# Joining and Appending DataFrames in PySpark HW

Now it's time to test your knowledge and further engrain the concepts we touched on in the lectures. Let's go ahead and get started.




**As always let's start our Spark instance.**

In [1]:
# import findspark
# findspark.init()

import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
# May take awhile locally
spark = SparkSession.builder.appName("joins").getOrCreate()

cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
print("You are working with", cores, "core(s)")
spark

You are working with 1 core(s)


## Read in the database

Let cotinue working with our college courses dataframe to get some more insights and practice what we have learned!Let's read in the whole database using the loop function that we learned about in the lecture to automatically read in all the datasets from the uw-madision-courses folder (there are too many datasets to each one individually.

In [9]:
import os

path = "Datasets/uw-madison-courses/"

df_list = []
for filename in os.listdir(path):
    if filename.endswith(".csv"):
        filename_list = filename.split(".") #separate path from .csv
        df_name = filename_list[0]
        df = spark.read.csv(path+filename,inferSchema=True,header=True)
        df.name = df_name
        df_list.append(df_name)
        exec(df_name + ' = df')

Full list of dfs:
['subjects', 'subject_memberships', 'rooms', 'schedules', 'sections', 'courses', 'course_offerings', 'instructors', 'teachings', 'grade_distributions']


Now check the contents of a few of the dataframses that were read in above.

In [11]:
print("Full list of dfs:")
print(df_list)

Full list of dfs:
['subjects', 'subject_memberships', 'rooms', 'schedules', 'sections', 'courses', 'course_offerings', 'instructors', 'teachings', 'grade_distributions']


## Recap: About this database

You will notice that there are several more tables in the uw-madision-courses folder than there are read in above. This so that you will have a chance to practice your own custom joins and learn about the relationships between a real database work. Sometimes we don't know how they are related and we need to figure it out! I'll save that for the HW :) 

Here is a look at some of the important variables we can use to join our tables:

 - course_offerings: uuid, course_uuid, term_code, name
 - instructors: id, name
 - schedules: uuid
 - sections: uuid, course_offering_uuid,room_uuid, schedule_uuid
 - teachings: instructor_id, section_uuid
 - courses: uuid
 - grade_distributions: course_offering_uuid,section_number
 - rooms: uuid, facility_code, room_code
 - subjects: code
 - subject_memberships: subject_code, course_offering_uuid
 
 **Source:** https://www.kaggle.com/Madgrades/uw-madison-courses
 
So alright, let's use this information to discover some insights from this data!

## 1a. Can you assign the room numbers to each section of each course?

Show only the rooms uuid, facility code, room number, term code and the name of the course from the course_offerings table.

In [15]:
sections.toPandas().head(1)

,uuid,course_offering_uuid,section_type,number,room_uuid,schedule_uuid
0,45adf63c-48c9-3659-8561-07556d2d4ddf,344b3ebe-da7e-314c-83ed-9425269695fd,FLD,1,null,f2d66a4d-0c08-3b48-abf6-649fffd7ae90


In [21]:
step1 = course_offerings.join(sections, sections.course_offering_uuid == course_offerings.uuid, how='left')
step1.limit(5).toPandas()

,uuid,course_uuid,term_code,name,uuid,course_offering_uuid,section_type,number,room_uuid,schedule_uuid
0,f39d2078-0be1-3dbf-a696-189fe822b889,b9b9c9ef-3fed-3beb-8c16-fd39345e2aaf,1162,Foundations of Spec Education,51a42001-518b-3174-aa3a-4fb212d4461d,f39d2078-0be1-3dbf-a696-189fe822b889,LEC,1,7a70c303-e14c-3fee-a272-db3fac23dec4,dd049678-0cd2-3506-917b-74b987a0bc02
1,8952021c-516a-3116-81f0-ac3c902802ca,8776518d-d24c-329d-aae9-15f98069372e,1182,Theories-Soc & Ed Change,665cbb4b-91f5-3e7a-b727-e6a59933d9c8,8952021c-516a-3116-81f0-ac3c902802ca,LEC,1,15933a67-a696-396d-a6da-23976a9233a8,28acce08-7f70-3f99-b67f-a0fbeb53682f
2,344b3ebe-da7e-314c-83ed-9425269695fd,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1092,Cooperative Education Prog,45adf63c-48c9-3659-8561-07556d2d4ddf,344b3ebe-da7e-314c-83ed-9425269695fd,FLD,1,null,f2d66a4d-0c08-3b48-abf6-649fffd7ae90
3,d019e707-72e7-38e9-a41f-099e1a8af89e,8776518d-d24c-329d-aae9-15f98069372e,1162,Theories-Soc & Ed Change,bf89b630-2afb-37ac-9eae-acd34ac1156b,d019e707-72e7-38e9-a41f-099e1a8af89e,LEC,1,33728ee2-761a-3019-9ec6-45bd2df0d58e,ce0f2b37-84b3-351a-b739-7563b12d4bce
4,10948e6e-c96f-338b-ad62-336236b2b160,1adff8c1-9b21-3229-b2d0-a900189661f1,1104,Prosem-S E Asian History,9554b15b-4f4a-394e-841a-d2609df677d4,10948e6e-c96f-338b-ad62-336236b2b160,SEM,2,31837044-f493-3f6f-a365-41a4cd94a1c9,8ba7e8c1-aff2-35f6-b412-18b071028536


In [22]:
step2 = step1.join(rooms, step1.room_uuid == rooms.uuid)
step2.limit(5).toPandas()

,uuid,course_uuid,term_code,name,uuid,course_offering_uuid,section_type,number,room_uuid,schedule_uuid,uuid,facility_code,room_code
0,ea3b717c-d66b-30dc-8b37-964d9688295f,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1172,Cooperative Education Prog,9395dc21-15d1-3fab-8d1f-6f3fe6114c48,ea3b717c-d66b-30dc-8b37-964d9688295f,FLD,1,04368a56-c959-3e4b-8b3d-f4cc3538fea5,f2d66a4d-0c08-3b48-abf6-649fffd7ae90,04368a56-c959-3e4b-8b3d-f4cc3538fea5,OFF CAMPUS,null
1,ea3b717c-d66b-30dc-8b37-964d9688295f,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1172,Cooperative Education Prog,da41b0aa-2b81-378f-914e-39f3465eed19,ea3b717c-d66b-30dc-8b37-964d9688295f,FLD,1,04368a56-c959-3e4b-8b3d-f4cc3538fea5,f2d66a4d-0c08-3b48-abf6-649fffd7ae90,04368a56-c959-3e4b-8b3d-f4cc3538fea5,OFF CAMPUS,null
2,075da420-5f49-3dd0-93df-13e3c152e1b1,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1114,Cooperative Education Prog,51c4dc00-1fc7-3c7c-98a1-eba112aa3e5d,075da420-5f49-3dd0-93df-13e3c152e1b1,FLD,1,04368a56-c959-3e4b-8b3d-f4cc3538fea5,f2d66a4d-0c08-3b48-abf6-649fffd7ae90,04368a56-c959-3e4b-8b3d-f4cc3538fea5,OFF CAMPUS,null
3,2b4e216d-a728-3713-8c7c-19afffc6b2fd,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1104,Cooperative Education Prog,8fc362a6-d94e-3ade-aee7-87796c567dcf,2b4e216d-a728-3713-8c7c-19afffc6b2fd,FLD,1,04368a56-c959-3e4b-8b3d-f4cc3538fea5,f2d66a4d-0c08-3b48-abf6-649fffd7ae90,04368a56-c959-3e4b-8b3d-f4cc3538fea5,OFF CAMPUS,null
4,ff511882-5eab-3e7b-a89f-8fbfd1906127,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1134,Cooperative Education Prog,b21564d2-2bc4-3b86-aa10-a4a3130eaebe,ff511882-5eab-3e7b-a89f-8fbfd1906127,FLD,1,04368a56-c959-3e4b-8b3d-f4cc3538fea5,f2d66a4d-0c08-3b48-abf6-649fffd7ae90,04368a56-c959-3e4b-8b3d-f4cc3538fea5,OFF CAMPUS,null


## 1b. Now show same output as above but for only facility number 0469 (facility_code)

In [29]:
step2.filter(step2.facility_code=="0469").limit(5).toPandas()

,uuid,course_uuid,term_code,name,uuid,course_offering_uuid,section_type,number,room_uuid,schedule_uuid,uuid,facility_code,room_code
0,bd5baa02-e0ca-39df-a61d-361e8534f398,eb1a8a08-a675-3c45-aa49-23a6589fed56,1152,Fundamentals-Flute,794937a3-677f-3b34-a1c9-8a7743e6b620,bd5baa02-e0ca-39df-a61d-361e8534f398,LAB,1,9759cb5f-a7d3-3d0c-b8b3-03910bd0153e,76c82895-6420-3a2c-bb27-5b19b2e07755,9759cb5f-a7d3-3d0c-b8b3-03910bd0153e,0469,2441
1,2101e24a-f58f-33d1-8ca1-61778a83f886,eb1a8a08-a675-3c45-aa49-23a6589fed56,1092,Fundamentals-Flute,7f28f121-ae3a-3312-8c21-5b4eb0051922,2101e24a-f58f-33d1-8ca1-61778a83f886,LAB,1,9759cb5f-a7d3-3d0c-b8b3-03910bd0153e,76c82895-6420-3a2c-bb27-5b19b2e07755,9759cb5f-a7d3-3d0c-b8b3-03910bd0153e,0469,2441
2,821927cc-c5eb-38c4-93f4-bcca6ee066b1,eb1a8a08-a675-3c45-aa49-23a6589fed56,1172,Fundamentals-Flute,b04fffcf-cb7c-3483-87b4-9b0790d2794f,821927cc-c5eb-38c4-93f4-bcca6ee066b1,LAB,1,6af80b0b-b3e3-370a-925f-829869f007bc,76c82895-6420-3a2c-bb27-5b19b2e07755,6af80b0b-b3e3-370a-925f-829869f007bc,0469,4411
3,a35e69d8-4355-3bf7-83f7-21f09212cb41,eb1a8a08-a675-3c45-aa49-23a6589fed56,1162,Fundamentals-Flute,a818c608-0420-342e-8042-6f4b3b6b8e01,a35e69d8-4355-3bf7-83f7-21f09212cb41,LAB,1,9759cb5f-a7d3-3d0c-b8b3-03910bd0153e,76c82895-6420-3a2c-bb27-5b19b2e07755,9759cb5f-a7d3-3d0c-b8b3-03910bd0153e,0469,2441
4,3e551eb9-a026-3ebb-98e9-3d4391c3bafd,eb1a8a08-a675-3c45-aa49-23a6589fed56,1132,Fundamentals-Flute,4711c087-1025-36dc-b379-e0f96f2cbc57,3e551eb9-a026-3ebb-98e9-3d4391c3bafd,LAB,1,9759cb5f-a7d3-3d0c-b8b3-03910bd0153e,76c82895-6420-3a2c-bb27-5b19b2e07755,9759cb5f-a7d3-3d0c-b8b3-03910bd0153e,0469,2441


## 2. Count how many sections are offered for each subject for each facility

*Note: this will involve a groupby*

In [72]:
step3 = step2.join(subject_memberships, subject_memberships.course_offering_uuid == step2.course_offering_uuid, how='left')
step3.limit(5).toPandas()

,uuid,course_uuid,term_code,name,uuid,course_offering_uuid,section_type,number,room_uuid,schedule_uuid,uuid,facility_code,room_code,subject_code,course_offering_uuid
0,ea3b717c-d66b-30dc-8b37-964d9688295f,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1172,Cooperative Education Prog,9395dc21-15d1-3fab-8d1f-6f3fe6114c48,ea3b717c-d66b-30dc-8b37-964d9688295f,FLD,1,04368a56-c959-3e4b-8b3d-f4cc3538fea5,f2d66a4d-0c08-3b48-abf6-649fffd7ae90,04368a56-c959-3e4b-8b3d-f4cc3538fea5,OFF CAMPUS,null,240,ea3b717c-d66b-30dc-8b37-964d9688295f
1,ea3b717c-d66b-30dc-8b37-964d9688295f,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1172,Cooperative Education Prog,9395dc21-15d1-3fab-8d1f-6f3fe6114c48,ea3b717c-d66b-30dc-8b37-964d9688295f,FLD,1,04368a56-c959-3e4b-8b3d-f4cc3538fea5,f2d66a4d-0c08-3b48-abf6-649fffd7ae90,04368a56-c959-3e4b-8b3d-f4cc3538fea5,OFF CAMPUS,null,418,ea3b717c-d66b-30dc-8b37-964d9688295f
2,ea3b717c-d66b-30dc-8b37-964d9688295f,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1172,Cooperative Education Prog,9395dc21-15d1-3fab-8d1f-6f3fe6114c48,ea3b717c-d66b-30dc-8b37-964d9688295f,FLD,1,04368a56-c959-3e4b-8b3d-f4cc3538fea5,f2d66a4d-0c08-3b48-abf6-649fffd7ae90,04368a56-c959-3e4b-8b3d-f4cc3538fea5,OFF CAMPUS,null,490,ea3b717c-d66b-30dc-8b37-964d9688295f
3,ea3b717c-d66b-30dc-8b37-964d9688295f,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1172,Cooperative Education Prog,9395dc21-15d1-3fab-8d1f-6f3fe6114c48,ea3b717c-d66b-30dc-8b37-964d9688295f,FLD,1,04368a56-c959-3e4b-8b3d-f4cc3538fea5,f2d66a4d-0c08-3b48-abf6-649fffd7ae90,04368a56-c959-3e4b-8b3d-f4cc3538fea5,OFF CAMPUS,null,636,ea3b717c-d66b-30dc-8b37-964d9688295f
4,ea3b717c-d66b-30dc-8b37-964d9688295f,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1172,Cooperative Education Prog,9395dc21-15d1-3fab-8d1f-6f3fe6114c48,ea3b717c-d66b-30dc-8b37-964d9688295f,FLD,1,04368a56-c959-3e4b-8b3d-f4cc3538fea5,f2d66a4d-0c08-3b48-abf6-649fffd7ae90,04368a56-c959-3e4b-8b3d-f4cc3538fea5,OFF CAMPUS,null,207,ea3b717c-d66b-30dc-8b37-964d9688295f


In [73]:
print(step3.printSchema())

root
 |-- uuid: string (nullable = true)
 |-- course_uuid: string (nullable = true)
 |-- term_code: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- uuid: string (nullable = true)
 |-- course_offering_uuid: string (nullable = true)
 |-- section_type: string (nullable = true)
 |-- number: integer (nullable = true)
 |-- room_uuid: string (nullable = true)
 |-- schedule_uuid: string (nullable = true)
 |-- uuid: string (nullable = true)
 |-- facility_code: string (nullable = true)
 |-- room_code: string (nullable = true)
 |-- subject_code: integer (nullable = true)
 |-- course_offering_uuid: string (nullable = true)

None


In [75]:
# filter facility is not off-campus or online
step3 = step3.filter(step3.facility_code.isin(["OFF CAMPUS","ONLINE"]) == False).select(["facility_code", "subject_code"])
step3.limit(5).toPandas()

,facility_code,subject_code
0,0032,741
1,0032,741
2,0032,741
3,0032,741
4,0032,741


In [76]:
step4 = step3.join(subjects, subjects.code == step3.subject_code, how='left').select(["name", "facility_code"])
step4.limit(5).toPandas()

,name,facility_code
0,Dance,0032
1,Dance,0032
2,Dance,0032
3,Dance,0032
4,Dance,0032


In [77]:
step4.groupby("name").pivot("facility_code").count().limit(10).toPandas()

,name,0000,0011,0015,0018,0021,0025,0028,0031,0032,...,1400F,1400G,1400K,1404K,1405E,1405G,1406F,1406K,1480,1485
0,Asian American Studies,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,28.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Naval Science,NaN,NaN,NaN,NaN,NaN,55.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Religious Studies,NaN,NaN,NaN,24.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HEBREW,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Dairy Science,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Nutritional Sciences,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Botany,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Urban and Regional Planning,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Cell and Regenerative Biology,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,4.0
9,WILDLIFE ECOLOGY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 3. What are the hardest classes?

Let's see if we can figure out which classes are the hardest by seeing how many students failed. Note that you will first need to aggregate the grades table by the course uuid to include all sections. Show the name of the course as well that you will need to get from the course_offering table.

In [84]:
# first need to group total fails for one course
grades1 = grade_distributions.groupby("course_offering_uuid").sum("f_count")
grades1.limit(5).toPandas()

,course_offering_uuid,sum(f_count)
0,c939dd5e-43ba-3266-8f74-573f629de75b,0
1,75fdf27b-0e52-3544-96eb-d594a5ad969f,0
2,cfeba94d-8e0e-320b-a904-ea2c2a31c000,7
3,ceabe145-78e5-33c2-9b68-3a1eea9c2864,4
4,1ee15e7e-fa91-3205-8999-4cac58a95b61,0


In [89]:
from pyspark.sql.functions import *

In [90]:
grades2 = course_offerings.join(grades1, grade_distributions.course_offering_uuid == course_offerings.uuid, how='left').select(["name", "sum(f_count)"]).orderBy(col("sum(f_count)").desc())

In [92]:
grades2.limit(1).toPandas()

,name,sum(f_count)
0,Calc--Functns of Variables,72


## Challenge Question: Automating data entry errors

We see in the dataframe below that there are several typos of various animal names. If this was a large database of several millions of records, correcting these errors would be way too labor intensive. How can we automate correcting these errors?

*Hint: Leven...*

In [93]:
values = [('Monkey',10),('Monkay',36),('Mnky',123), \
          ('Elephant',48),('Elefant',16),('Ellafant',1), \
          ('Hippopotamus',48),('Hipopotamus',16),('Hippo',1)]
zoo = spark.createDataFrame(values,['Animal','age'])
zoo.show()

+------------+---+
|      Animal|age|
+------------+---+
|      Monkey| 10|
|      Monkay| 36|
|        Mnky|123|
|    Elephant| 48|
|     Elefant| 16|
|    Ellafant|  1|
|Hippopotamus| 48|
| Hipopotamus| 16|
|       Hippo|  1|
+------------+---+



In [95]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

# first we create a dataframe with the 3 options we want to choose from
options = spark.createDataFrame(['Monkey', 'Elephant', 'Hippopotamus'], StringType())
options.show()

+------------+
|       value|
+------------+
|      Monkey|
|    Elephant|
|Hippopotamus|
+------------+



In [97]:
# then we join two dataframes together with a levenshtein condition >5
# Levenshtein distance is used to compare two strings to find how different they are. 
# The diff is calculated based on the number of edits (insertion, deletion or substitutions) required. 
results = zoo.join(options, levenshtein(zoo["Animal"], options["value"])<5, 'left')
results.show()

+------------+---+------------+
|      Animal|age|       value|
+------------+---+------------+
|      Monkey| 10|      Monkey|
|      Monkay| 36|      Monkey|
|        Mnky|123|      Monkey|
|    Elephant| 48|    Elephant|
|     Elefant| 16|    Elephant|
|    Ellafant|  1|    Elephant|
|Hippopotamus| 48|Hippopotamus|
| Hipopotamus| 16|Hippopotamus|
|       Hippo|  1|        null|
+------------+---+------------+



### Great job! 